In [2]:
# Imports
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
import dask.dataframe as ddf
from datetime import datetime, timedelta

Reducing the dataset: Using the columns identified in null analysis and selecting only specific Seagate models

In [3]:
parquet_files = glob.glob('C:/com748/data/raw/parquet/*.parquet')

for file in parquet_files:

    date = Path(file).stem

    # Get columns with low null percentage
    df = pd.read_parquet(file, 
                            columns=['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'smart_1_normalized', 'smart_1_raw', 
                                   'smart_3_normalized', 'smart_3_raw', 'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized', 'smart_5_raw', 
                                   'smart_7_normalized', 'smart_7_raw', 'smart_9_normalized', 'smart_9_raw', 'smart_10_normalized', 'smart_10_raw', 
                                   'smart_12_normalized', 'smart_12_raw', 'smart_187_normalized', 'smart_187_raw', 'smart_188_normalized', 'smart_188_raw', 
                                   'smart_190_normalized', 'smart_190_raw', 'smart_192_normalized', 'smart_192_raw', 'smart_193_normalized', 'smart_193_raw', 
                                   'smart_194_normalized', 'smart_194_raw', 'smart_197_normalized', 'smart_197_raw', 'smart_198_normalized', 'smart_198_raw', 
                                   'smart_199_normalized', 'smart_199_raw', 'smart_240_normalized', 'smart_240_raw', 'smart_241_normalized', 'smart_241_raw', 
                                   'smart_242_normalized', 'smart_242_raw'], 
                            engine='pyarrow')
    
    df = df[df['model'].isin(['ST4000DM000', 'ST12000NM0007', 'ST8000NM0055', 'ST3000DM001', 'ST12000NM0008', 'ST8000DM002', 'ST14000NM001G'])]

    parquet_path = f'C:/com748/data/processed/daily/{date}.parquet'
    table_data = pa.Table.from_pandas(df)
    pq.write_table(table_data, parquet_path)


Collect failure SMART data for various lookahead windows (days):

# Day minus 0 (last day drive was operational before failing)

In [21]:
# Day minus 0 (last day drive was operational before failing)
parquet_files = glob.glob('C:/com748/data/processed/daily/*.parquet')

dataframes_list = []

for file in reversed(parquet_files):
    date = Path(file).stem
    df = ddf.read_parquet(file, ignore_metadata_file=True, engine='pyarrow')

    df_failures = df[df['failure'] == 1]
    df_failures['file_date'] = date

    dataframes_list.append(df_failures.compute())

output_df = pd.concat(dataframes_list, ignore_index=True)

# Write output to file
output_df.to_csv('C:/com748/code/com748/data/processed/day_minus_0/failures.csv', index=False)

C:\Users\ali_m\AppData\Local\Temp\ipykernel_43912\920707536.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat(dataframes_list, ignore_index=True)


# Define function to retrieve day minus n data:

In [3]:
def get_day_minus_n_data(n, date, serial_number):

    # Convert string to datetime
    date_dt = datetime.strptime(date, '%Y-%m-%d')
    # Get day minus n date
    date_dt_minus_n = date_dt - timedelta(days=n)
    # Convert dates back to String
    day_minus_n = date_dt_minus_n.strftime('%Y-%m-%d')

    # Read day_minus_n file filtered by serial_number
    try:
        df_day_minus_n = ddf.read_parquet(f'C:/com748/data/processed/daily/{day_minus_n}.parquet', engine='pyarrow', filters=[("serial_number", "==", f"{serial_number}")])
        df_day_minus_n = df_day_minus_n.compute()
        if len(df_day_minus_n.index) > 0: 
            return df_day_minus_n.iloc[0]
        else: 
            return None
    except:
        print(f'Could not find day_minus_n file ({day_minus_n})')
        return None

# Day minus 1 (1 day before last operational day)

In [34]:
# Read day_minus_0 failures data
df_day_minus_0 = pd.read_csv('C:/com748/code/com748/data/processed/day_minus_0/failures.csv')

# iterate through day_minus_0 failures
df_day_minus_1 = df_day_minus_0.apply(lambda row: get_day_minus_n_data(1, row['file_date'], row['serial_number']), axis=1)
null_count = len(df_day_minus_1[df_day_minus_1['serial_number'].isna()].index)
print(f'Total failures where day_minus_1 data not found: {null_count}')

# Drop null rows
df_day_minus_1 = df_day_minus_1[df_day_minus_1['serial_number'].notna()]

# Write data to file
df_day_minus_1.to_csv('C:/com748/code/com748/data/processed/day_minus_1/failures.csv', index=False)


Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Total failures where day_minus_1 data not found: 297


# Day minus 2 (2 days before last operational day)

In [4]:
# Read day_minus_0 failures data
df_day_minus_0 = pd.read_csv('C:/com748/code/com748/data/processed/day_minus_0/failures.csv')

# iterate through day_minus_0 failures
df_day_minus_2 = df_day_minus_0.apply(lambda row: get_day_minus_n_data(2, row['file_date'], row['serial_number']), axis=1)
null_count = len(df_day_minus_2[df_day_minus_2['serial_number'].isna()].index)
print(f'Total failures where day_minus_2 data not found: {null_count}')

# Drop null rows
df_day_minus_2 = df_day_minus_2[df_day_minus_2['serial_number'].notna()]

# Write data to file
df_day_minus_2.to_csv('C:/com748/code/com748/data/processed/day_minus_2/failures.csv', index=False)


Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-30)
Could not find day_minus_n file (2013-12-30)
Could not find day_minus_n file (2013-12-30)
Total failures where day_minus_2 data not found: 309


# Day minus 7 (7 days before last operational day)

In [5]:
# Read day_minus_0 failures data
df_day_minus_0 = pd.read_csv('C:/com748/code/com748/data/processed/day_minus_0/failures.csv')

# iterate through day_minus_0 failures
df_day_minus_7 = df_day_minus_0.apply(lambda row: get_day_minus_n_data(7, row['file_date'], row['serial_number']), axis=1)
null_count = len(df_day_minus_7[df_day_minus_7['serial_number'].isna()].index)
print(f'Total failures where day_minus_7 data not found: {null_count}')

# Drop null rows
df_day_minus_7 = df_day_minus_7[df_day_minus_7['serial_number'].notna()]

# Write data to file
df_day_minus_7.to_csv('C:/com748/code/com748/data/processed/day_minus_7/failures.csv', index=False)


Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-31)
Could not find day_minus_n file (2013-12-26)
Could not find day_minus_n file (2013-12-26)
Could not find day_minus_n file (2013-12-26)
Could not find day_minus_n file (2013-12-26)
Could not find day_minus_n file (2013-12-26)
Could not find day_minus_n file (2013-12-26)
Could not find day_minus_n file (2013-12-25)
Could not find day_minus_n file (2013-12-25)
Could not find day_minus_n file (2013-12-25)
Total failures where day_minus_7 data not found: 255
